In [ ]:
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from zumi.zumi import Zumi
import cv2
import IPython.display
import PIL.Image
from time import sleep
from zumi.zumi import Zumi

car_path = "car.xml"
#credit https://github.com/andrewssobral/vehicle_detection_haarcascades
cascade = cv2.CascadeClassifier(car_path)

#height and width of the camera in pixels
width = 160
height = 120

screen = Screen()
zumi = Zumi()

screen.blink()

def error_drive(error,k = 0.5):
    #make sure max error is 80
    iterations = abs(int(error*k))
    max_iter = 12
    
    if iterations>max_iter:
        iterations = max_iter
    #if zumi is on the left or right of the object
    if error > 0 and iterations > 1:
        print("left")
        for i in range(iterations):
            zumi.control_motors(i,-i)

    elif error < 0 and iterations > 1:
        print("right")
        for i in range(iterations):
            zumi.control_motors(-i,i)  
    zumi.stop()        
    #print("centered error",error , "iterations",iterations)    

        
try:
    error = 0
    camera = Camera(width,height)
    camera.start_camera()
    while(1):

        frame = camera.capture()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        signs = cascade.detectMultiScale(gray, scaleFactor=1.05, 
                                         minNeighbors=8, minSize=(10,10), flags=cv2.CASCADE_SCALE_IMAGE)
        for (x,y,w,h) in signs:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
            cv2.putText(frame, "Car", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (100,255,155), 1, cv2.LINE_AA) 
            error = width/2-x
            error_drive(error)
            break
        if len(signs) == 0:
            zumi.stop()
            
        #display the image
        IPython.display.display(PIL.Image.fromarray(frame))
        #clear the output for the next frame
        IPython.display.clear_output(wait=True)
        
finally:
    camera.close()

In [ ]:
zumi.stop()